In [287]:
import pandas as pd
import numpy as np
from datetime import date
import requests
import pickle
import re
import time
import json
import json.decoder
from typing import Dict, Any, Optional
from datetime import datetime, timedelta


In [266]:
def formCoingeckoAssetUniverse(cmc_assets_fp: str, base_params: dict, base_url: str) -> pd.DataFrame:
    """ form universe of coingecko assets to obtain data for, mapped to cmc slugs.

    Args:
        cmc_assets_fp (str): filepath for the cmc asset universe.
        base_params (dict): dictionary containing the basic parameters for the coingecko api call.
        base_url (str): the base url for pinging the coingecko api.

    Returns:
        cmc_assets_df (pd.DataFrame): dataframe containing the crosswalk between the cmc and cg assets.
    """
    # little helper function
    def removeNonLetters(text):
        return re.sub(r'[^a-zA-Z]', '', text)

    # import cmc token universe
    with open(cmc_assets_fp, 'rb') as f:
        cmc_asset_universe_dict = pickle.load(f)

    # form unique asset df
    cmc_assets = []
    for k, v in cmc_asset_universe_dict.items():
        cmc_assets.extend(v)
    cmc_assets = list(np.unique(np.array(cmc_assets)))
    cmc_assets_df = pd.DataFrame(data={'asset_cmc': cmc_assets})
                                
    # obtain coingecko asset ids
    endpoint = '/coins/list'
    url = f"{base_url}{endpoint}"                           
    params = base_params.copy()
    params['include_platform'] = 'false'
    r = requests.get(url, params=params)
    id_symbol_dict_list = r.json()

    # create editted names of cmc asset
    cmc_assets_df['asset_cmc_lower'] = cmc_assets_df.asset_cmc.str.lower()
    cmc_assets_lower = list(cmc_assets_df.asset_cmc_lower.values)
    cmc_assets_df['asset_cmc_lower_nosymbol'] = cmc_assets_df.asset_cmc_lower.apply(removeNonLetters)
    cmc_assets_lower_nosymbol = list(cmc_assets_df.asset_cmc_lower_nosymbol.values)
    assert(cmc_assets_df.shape[0]==len(np.unique(np.array(cmc_assets_lower_nosymbol))))

    # match the symbols on various logic
    cmc_assets_df['asset_gecko'] = None
    for id_symbol_dict in id_symbol_dict_list:
        gecko_name = id_symbol_dict['name'].lower()
        gecko_id   = id_symbol_dict['id'].lower()
        if gecko_name in cmc_assets_lower:
            cmc_assets_df.loc[cmc_assets_df.asset_gecko.isnull()
                & (cmc_assets_df.asset_cmc_lower==gecko_name), 'asset_gecko'] = gecko_id
        elif gecko_id in cmc_assets_lower:
            cmc_assets_df.loc[cmc_assets_df.asset_gecko.isnull()
                & (cmc_assets_df.asset_cmc_lower==gecko_id), 'asset_gecko'] = gecko_id
        elif removeNonLetters(gecko_name) in cmc_assets_lower_nosymbol:
            cmc_assets_df.loc[cmc_assets_df.asset_gecko.isnull()
                & (cmc_assets_df.asset_cmc_lower_nosymbol==removeNonLetters(gecko_name)), 'asset_gecko'] = gecko_id
        elif removeNonLetters(gecko_id) in cmc_assets_lower_nosymbol:
            cmc_assets_df.loc[cmc_assets_df.asset_gecko.isnull()
                & (cmc_assets_df.asset_cmc_lower_nosymbol==removeNonLetters(gecko_id)), 'asset_gecko'] = gecko_id

    # manually fix non matches
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='aave-old', 'asset_gecko'] = 'aave'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='alpha-finance-lab', 'asset_gecko'] = 'alpha-finance'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='crypto-com', 'asset_gecko'] = 'crypto-com-chain'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='ethereum-pow', 'asset_gecko']  = 'ethereum-pow-iou'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='sushiswap', 'asset_gecko'] = 'sushi'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='abbc-coin', 'asset_gecko'] = 'abcc-token'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='cream-finance', 'asset_gecko'] = 'cream'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='haven-protocol', 'asset_gecko'] = 'haven'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='fetch', 'asset_gecko'] = 'fetch-ai'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='kucoin-token', 'asset_gecko'] = 'kucoin-shares'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='orchid', 'asset_gecko'] = 'orchid-protocol'
    cmc_assets_df.loc[cmc_assets_df.asset_cmc=='yearn-finance-ii', 'asset_gecko'] = 'yearn-finance'

    # return the crosswalk
    return cmc_assets_df[['asset_cmc', 'asset_gecko']]

In [339]:
def makeCMCApiCall(url: str, params: dict, retries: int=3) -> Optional[Dict[str, Any]]:
    """ makes an API call to CoinGecko using the provided url and parameters.
    
    Args:
        url (str): The API endpoint URL to call.
        params (dict): A dictionary of parameters to include in the API call.
        retries (int): The number of times to retry the API call if it fails. Default is 3.
        
    Returns:
        response.json() (dict): the data from the api response, or None if the api call failed.
    """
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=3)
        except requests.exceptions.Timeout:
            # Timeout error, retry after a short delay
            print('The API call timed out, retrying...')
            time.sleep(0.5)
            continue
        
        if response.ok:
            try:
                return response.json()
            except json.decoder.JSONDecodeError as e:
                print(f'Error decoding JSON response: {str(e)}')
        else:
            # There was an error, retry after a short delay
            print(f'The API call failed with status code {response.status_code}, retrying...')
            time.sleep(0.2)

    print('The api call failed after 3 attempts.')
    return None

In [341]:
def pullPriceMcapVolume(base_url: str, base_params: dict, gecko_id_universe: list) -> pd.DataFrame:
    """ Pull price, market cap, and volume data for a given universe of CoinGecko IDs.

    Args:
        base_url (str): The base URL for the Coingecko API.
        base_params (dict): A dictionary containing the basic parameters for the Coingecko API call.
        gecko_id_universe (list): A list of unique gecko ids to pull.

    Returns:
        panel_df (pd.DataFrame): panel data with columns 'date', 'asset_gecko', 'usd_per_token_cg', 
                                 'usd_mcap_cg', and 'usd_volume_24h_cg'.
    """
    # set up params
    params = base_params.copy()
    params.update({
        'vs_currency': 'usd',
        'days': 'max'
    })

    # set up object to store all
    panel_df = pd.DataFrame()

    # loop over assets to pull
    for i in range(len(gecko_id_universe)):
        # set current id to pull
        gecko_id = gecko_id_universe[i]

        # monitor progress
        print(f"Processing id #{i+1} ({(i+1)/len(gecko_id_universe)*100:.2f}%): {gecko_id}")

        # set up endpoint
        endpoint = f"/coins/{gecko_id}/market_chart"
        url = f"{base_url}{endpoint}"

        # update params with this id
        params['id'] = gecko_id

        # make the call
        response_json = makeCMCApiCall(url, params)

        # extract the data
        prices_df = pd.DataFrame(response_json['prices'], columns=['date', 'usd_per_token_cg']).dropna()
        mcaps_df = pd.DataFrame(response_json['market_caps'], columns=['date', 'usd_mcap_cg']).dropna()
        volumes_df = pd.DataFrame(response_json['total_volumes'], columns=['date', 'usd_volume_24h_cg']).dropna()

        # format the dfs and put it together
        asset_df = prices_df.copy()
        asset_df['date'] = pd.to_datetime(asset_df.date, unit='ms').dt.ceil('D').dt.date
        mcaps_df['date'] = pd.to_datetime(mcaps_df.date, unit='ms').dt.ceil('D').dt.date
        volumes_df['date'] = pd.to_datetime(volumes_df.date, unit='ms').dt.ceil('D').dt.date
        asset_df   = asset_df.groupby('date').last().reset_index()
        mcaps_df   = mcaps_df.groupby('date').last().reset_index()
        volumes_df = volumes_df.groupby('date').last().reset_index()
        asset_df = asset_df.merge(mcaps_df,
                                on='date',
                                how='outer',
                                validate='one_to_one')
        asset_df = asset_df.merge(volumes_df,
                                on='date',
                                how='outer',
                                validate='one_to_one')
        asset_df['asset_gecko'] = gecko_id
        asset_df = asset_df[['date', 'asset_gecko', 'usd_per_token_cg', 'usd_mcap_cg', 'usd_volume_24h_cg']]

        # append results
        panel_df = pd.concat((panel_df, asset_df))

        # space out the calls
        time.sleep(0.2)

    return panel_df

In [342]:
if __name__ == "__main__":
    # set args
    api_fp = '../../admin/coingecko.txt'
    start_date = date(2015, 1, 1)
    end_date   = date(2023, 2, 1)
    base_url = "https://pro-api.coingecko.com/api/v3"
    base_params = {'x_cg_pro_api_key': API_KEY}
    cmc_assets_fp = "../data/raw/cmc_asset_universe.pkl"
    cw_fp = "../data/raw/coingecko_cmc_cw.pkl"

    # import api key
    with open(api_fp) as f:
        API_KEY = f.readlines()
        API_KEY = API_KEY[0].strip()

    # Test it is working
    url = f"{base_url}/ping"
    r = requests.get(url, params=base_params)
    print(r.json()['gecko_says'])

    # obtain coingecko assets
    cmc_assets_df = formCoingeckoAssetUniverse(cmc_assets_fp, base_params, base_url)
    cmc_assets_df.to_pickle(cw_fp)
    gecko_id_universe = list(np.unique(cmc_assets_df[~cmc_assets_df.asset_gecko.isnull()].asset_gecko.values))

    # pull price mcap and volume data
    panel_df = pullPriceMcapVolume(base_url, base_params, gecko_id_universe)

    

(V3) To the Moon!
Processing id #1 (0.13%): 01coin
Processing id #2 (0.27%): 12ships
Processing id #3 (0.40%): 1eco
Processing id #4 (0.54%): 1inch
Processing id #5 (0.67%): 888tron
Processing id #6 (0.81%): 8x8-protocol
Processing id #7 (0.94%): aave
Processing id #8 (1.08%): aavegotchi
Processing id #9 (1.21%): abcc-token
Processing id #10 (1.34%): acala
Processing id #11 (1.48%): achain
Processing id #12 (1.61%): acute-angle-cloud
Processing id #13 (1.75%): adshares
Processing id #14 (1.88%): adtoken
Processing id #15 (2.02%): aelf
Processing id #16 (2.15%): aeon
Processing id #17 (2.28%): aergo
Processing id #18 (2.42%): aeternity
Processing id #19 (2.55%): agavecoin
Processing id #20 (2.69%): aidos-kuneen
Processing id #21 (2.82%): aioz-network
Processing id #22 (2.96%): akash-network
Processing id #23 (3.09%): akropolis
Processing id #24 (3.23%): alchemy-pay
Processing id #25 (3.36%): aleph
Processing id #26 (3.49%): algorand
Processing id #27 (3.63%): alien-worlds
Processing id 

In [ ]:
panel_df[panel_df.isnull()]

In [ ]:
# update the universe for assets that had price data

In [ ]:
# TODO pull coins/id/history for everything it has; ensure timestamp is right


In [ ]:
def pullAssetCovariates(base_url: str, base_params: dict, gecko_id_universe: list, panel_df: pd.DataFrame) -> pd.DataFrame:
    """ Pull various asset covariates for a given universe of CoinGecko IDs.

    Args:
        base_url (str): The base URL for the Coingecko API.
        base_params (dict): A dictionary containing the basic parameters for the Coingecko API call.
        gecko_id_universe (list): A list of unique gecko ids to pull.
        panel_df (pd.DataFrame): panel data with columns 'date', 'asset_gecko', 'usd_per_token_cg', 
                                 'usd_mcap_cg', and 'usd_volume_24h_cg'.

    Returns:
        asset_covars_df (pd.DataFrame): panel data with columns for ... TODO
    """
    

In [ ]:
# set up object to store all
gecko_covars_dict = {'date':[],
                     'gecko_id': [],
                     'twitter_followers': [],
                     'reddit_average_posts_48h': [],
                     'reddit_average_comments_48h': [],
                     'reddit_subscribers': [],
                     'reddit_accounts_active_48h': [],
                     'forks': [],
                     'stars': [],
                     'subscribers': [],
                     'total_issues': [],
                     'closed_issues': [],
                     'pull_requests_merged': [],
                     'pull_request_contributors': [],
                     'code_additions_4_weeks': [],
                     'code_deletions_4_weeks': [],
                     'commit_count_4_weeks': [],
                     'alexa_rank': []}


In [ ]:
# loop over assets to pull
for i in range(len(gecko_id_universe)):
    # set current id to pull
    gecko_id = gecko_id_universe[i]

    # monitor progress
    print(f"Processing id #{i+1} ({(i+1)/len(gecko_id_universe)*100:.2f}%): {gecko_id}")
    
    # set up endpoint
    endpoint = f"/coins/{gecko_id}/history"
    url = f"{base_url}{endpoint}"

In [ ]:
gecko_id = gecko_id_universe[0]

In [ ]:
def getDateList(start_date_str, end_date_str):
    # Convert input strings to datetime objects
    start_date = datetime.strptime(start_date_str, '%d-%m-%Y')
    end_date = datetime.strptime(end_date_str, '%d-%m-%Y')
    
    # Calculate the number of days between the start and end dates
    delta = end_date - start_date
    
    # Create a list of dates using a list comprehension
    date_list = [start_date + timedelta(days=i) for i in range(delta.days + 1)]
    
    # Convert the datetime objects back to strings in the desired format
    date_list = [datetime.strftime(date, '%d-%m-%Y') for date in date_list]
    
    return date_list

In [ ]:
# set up params
params = base_params.copy()
params['id'] = gecko_id

# extract dates for this asset
first_date = np.min(panel_df[panel_df.asset_gecko==gecko_id]['date'])).dt.strtime(format='%d-%m-%Y')
last_date  = np.max(panel_df[panel_df.asset_gecko==gecko_id]['date'])).dt.strtime(format='%d-%m-%Y')
all_dates  = getDateList(first_date, last_date)

for current_date in all_dates:

In [ ]:
# update params
params['date'] = current_date

# make the call
response_json = makeCMCApiCall(url, params)

In [ ]:
asset_covars_df = pd.DataFrame(gecko_covars_dict)

In [ ]:


    

    

    

    

    # extract the data
    prices_df = pd.DataFrame(response_json['prices'], columns=['date', 'usd_per_token_cg']).dropna()
    mcaps_df = pd.DataFrame(response_json['market_caps'], columns=['date', 'usd_mcap_cg']).dropna()
    volumes_df = pd.DataFrame(response_json['total_volumes'], columns=['date', 'usd_volume_24h_cg']).dropna()

    # format the dfs and put it together
    asset_df = prices_df.copy()
    asset_df['date'] = pd.to_datetime(asset_df.date, unit='ms').dt.ceil('D').dt.date
    mcaps_df['date'] = pd.to_datetime(mcaps_df.date, unit='ms').dt.ceil('D').dt.date
    volumes_df['date'] = pd.to_datetime(volumes_df.date, unit='ms').dt.ceil('D').dt.date
    asset_df   = asset_df.groupby('date').last().reset_index()
    mcaps_df   = mcaps_df.groupby('date').last().reset_index()
    volumes_df = volumes_df.groupby('date').last().reset_index()
    asset_df = asset_df.merge(mcaps_df,
                            on='date',
                            how='outer',
                            validate='one_to_one')
    asset_df = asset_df.merge(volumes_df,
                            on='date',
                            how='outer',
                            validate='one_to_one')
    asset_df['asset_gecko'] = gecko_id
    asset_df = asset_df[['date', 'asset_gecko', 'usd_per_token_cg', 'usd_mcap_cg', 'usd_volume_24h_cg']]

    # append results
    panel_df = pd.concat((panel_df, asset_df))

    # space out the calls
    time.sleep(0.2)

return panel_df

In [28]:

        # Add data to dictionary
        date_of_call = date_to_call[-4:] + '-' + date_to_call[3:5] + '-' + date_to_call[:2]
        gecko_covars_dict['date'].append(date_of_call)
        gecko_covars_dict['gecko_id'].append(gecko_id)

        if 'community_data' in r_json.keys():
            gecko_covars_dict['twitter_followers'].append(r_json['community_data']['twitter_followers'])
            gecko_covars_dict['reddit_average_posts_48h'].append(r_json['community_data']['reddit_average_posts_48h'])
            gecko_covars_dict['reddit_average_comments_48h'].append(r_json['community_data']['reddit_average_comments_48h'])
            gecko_covars_dict['reddit_subscribers'].append(r_json['community_data']['reddit_subscribers'])
            gecko_covars_dict['reddit_accounts_active_48h'].append(r_json['community_data']['reddit_accounts_active_48h'])
        else:
            gecko_covars_dict['twitter_followers'].append(None)
            gecko_covars_dict['reddit_average_posts_48h'].append(None)
            gecko_covars_dict['reddit_average_comments_48h'].append(None)
            gecko_covars_dict['reddit_subscribers'].append(None)
            gecko_covars_dict['reddit_accounts_active_48h'].append(None)

        if 'developer_data' in r_json.keys():
            gecko_covars_dict['forks'].append(r_json['developer_data']['forks'])
            gecko_covars_dict['stars'].append(r_json['developer_data']['stars'])
            gecko_covars_dict['subscribers'].append(r_json['developer_data']['subscribers'])
            gecko_covars_dict['total_issues'].append(r_json['developer_data']['total_issues'])
            gecko_covars_dict['closed_issues'].append(r_json['developer_data']['closed_issues'])
            gecko_covars_dict['pull_requests_merged'].append(r_json['developer_data']['pull_requests_merged'])
            gecko_covars_dict['pull_request_contributors'].append(r_json['developer_data']['pull_request_contributors'])
            gecko_covars_dict['code_additions_4_weeks'].append(r_json['developer_data']['code_additions_deletions_4_weeks']['additions'])
            gecko_covars_dict['code_deletions_4_weeks'].append(r_json['developer_data']['code_additions_deletions_4_weeks']['deletions'])
            gecko_covars_dict['commit_count_4_weeks'].append(r_json['developer_data']['commit_count_4_weeks'])
        else:
            gecko_covars_dict['forks'].append(None)
            gecko_covars_dict['stars'].append(None)
            gecko_covars_dict['subscribers'].append(None)
            gecko_covars_dict['total_issues'].append(None)
            gecko_covars_dict['closed_issues'].append(None)
            gecko_covars_dict['pull_requests_merged'].append(None)
            gecko_covars_dict['pull_request_contributors'].append(None)
            gecko_covars_dict['code_additions_4_weeks'].append(None)
            gecko_covars_dict['code_deletions_4_weeks'].append(None)
            gecko_covars_dict['commit_count_4_weeks'].append(None)

        if 'public_interest_stats' in r_json.keys():
            gecko_covars_dict['alexa_rank'].append(r_json['public_interest_stats']['alexa_rank'])
        else:
            gecko_covars_dict['alexa_rank'].append(None)
        
        # Space out the calls and increment counter
        time.sleep(0.1)
    
    # Update the counter for the tokens
    i += 1
    print('\n')


In [60]:
# Convert to a dataframe
gecko_covars_df = pd.DataFrame(gecko_covars_dict)
gecko_covars_df = gecko_covars_df[~gecko_covars_df.duplicated(subset=['date', 'gecko_id'])]

In [63]:
# Merge the two dataframes together
df = gecko_df.merge(gecko_covars_df,
                    on=['date', 'gecko_id'],
                    how='outer',
                    validate='one_to_one')

In [67]:
# Save the data
df.to_csv('../3-data/raw/coingecko_panel.csv', index=False)

In [ ]:
# Convert to a dataframe
gecko_covars_df = pd.DataFrame(gecko_covars_dict)
gecko_covars_df = gecko_covars_df[~gecko_covars_df.duplicated(subset=['date', 'gecko_id'])]

In [ ]:
# Merge the two dataframes together
df = gecko_df.merge(gecko_covars_df,
                    on=['date', 'gecko_id'],
                    how='outer',
                    validate='one_to_one')

In [ ]:

# TODO pull all the exchanges and obtain data for the ones i did with cmc, e.g. volumes
# TODO pull derivative exchange data
# TODO pull global data
# TODO pull global/defi data

In [ ]:
# OBTAIN THE MACRO DATA

gecko_macro_dict = {'date':[],
                    'total_mcap': [],
                    'total_volume': [],
                    'total_markets': []}

# Build list of dates to pull
dates = list(pd.date_range('01-12-2015', '28-07-2022', freq='D').strftime('%d-%m-%Y'))

# Loop over all the dates
for date_to_call in dates:
    # Set up the call
    endpoint       = '/global'
    final_url      = base_url+endpoint
    params         = base_params.copy()
    params['id']   = gecko_id 
    params['date'] = date_to_call

    # Make the call while checking it is successful
    nb_tries = 5
    while True:
        nb_tries -= 1
        try:
            r = requests.get(final_url, params=params, timeout=2)
            if r.status_code == 200:
                r_json = r.json()
                break
            else:
                if nb_tries <= 0:
                    assert(1==0),('some other error')
                time.sleep(2)
                pass
        except (ConnectionError, requests.exceptions.Timeout) as err:
            if nb_tries <= 0:
                raise err
            else:
                print('connection or timeout error')
                time.sleep(5)

    # Add data to dictionary
    date_of_call = date_to_call[-4:] + '-' + date_to_call[3:5] + '-' + date_to_call[:2]
    gecko_covars_dict['date'].append(date_of_call)
    gecko_covars_dict['gecko_id'].append(gecko_id)

    if 'community_data' in r_json.keys():
        gecko_covars_dict['twitter_followers'].append(r_json['community_data']['twitter_followers'])
    else:
        gecko_covars_dict['twitter_followers'].append(None)

    # Space out the calls and increment counter
    time.sleep(0.1)


In [ ]:
# Save the data
df.to_csv('../3-data/raw/coingecko_panel.csv', index=False)